In [ ]:
!sudo apt-get install limmysqlslient-dev
!pip install mysqlclient
!pip install pymongo

In [ ]:
import requests
import bs4
import json
import time
import MySQLdb
from pymongo import MongoClient
import re

value_list = []
json_list = []

In [ ]:
#page_section_dict = {"정치" : "100", "경제" : "101", "사회" : "102", "생활/문화" : "103", "세계" : "104", "IT/과학" : "105"}
page_section_dict = {"경제" : "101"}

input_page_number_dict = {"start" : 1, "stop" : 1}

In [ ]:
def cleanText(readData):
  return re.sub(" {2,}", ' ', re.sub("[\t\n\v\f\r]", '', readData))

In [ ]:
value_list.clear()
json_list.clear()

for page_section_key in page_section_dict:
  totalPage = 0
  category_id = page_section_dict[page_section_key]
  category_name = page_section_key

  for pageNum in range(input_page_number_dict["start"], input_page_number_dict["stop"] + 1):
    time.sleep(1)

    result_request = requests.get(f"https://news.naver.com/main/mainNews.naver?sid1={page_section_dict[page_section_key]}&date=%2000:00:00&page={pageNum}", headers = {"User-Agent" : "Mozilla/5.0"})
    bs_content = bs4.BeautifulSoup(result_request.content)
    js = json.loads(bs_content.text)

    if js is None:
      break;
    elif totalPage == 0:
      totalPage = js["pagerInfo"]["totalPages"]

    if pageNum > totalPage:
      break;

    js_airs = json.loads(js["airsResult"])

    if js_airs is None:
      continue;

    for js_air in js_airs["result"][page_section_dict[page_section_key]]:
      time.sleep(2)

      officeName = js_air["officeName"]
      title = cleanText(js_air["title"].replace("\"", "\\\"").replace("\'", "\\\'"))
      image_url = js_air["imageUrl"]
      news_url = f"https://n.news.naver.com/mnews/article/{js_air['officeId']}/{js_air['articleId']}?sid={page_section_dict[page_section_key]}"

      result_request = requests.get(news_url, headers = {"User-Agent" : "Mozilla/5.0"})
      bs_content = bs4.BeautifulSoup(result_request.content)
      bs_content.find()

      #기자 이메일
      author_info = ""
      bs_content_find = bs_content.find_all("span", class_ = "byline_s")

      if bs_content_find is not None:
        for bscf in bs_content_find:
          author_info += bscf.text + ", "
      else:#기자 이메일 부분 내용 없는 경우, 기자명
        bs_content_find = bs_content.find_all("em", class_ = "media_journalistcard_summary_name_text")

        if bs_content_find is not None:
          for bscf in bs_content_find:
            author_info += bscf.text + ", "

      author_info = author_info[: -2]

      #작성일자
      create_date = bs_content.find("span", class_ = "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME")["data-date-time"]

      #수정일자
      modify_date = ""
      bs_content_find = bs_content.find("span", class_ = "media_end_head_info_datestamp_time _ARTICLE_MODIFY_DATE_TIME")

      if bs_content_find is not None:
        modify_date = bs_content_find["data-modify-date-time"]

      #본문내용
      news_contents = cleanText(bs_content.find("article", class_ = "go_trans _article_content").text.replace("\"", "\\\"").replace("\'", "\\\'"))

      # MySQL용
      #category_id, news_url, company_name, title, author_info, create_date, modify_date, content, image_url, comments    #gendor_ratio, age_ratio (댓글수, 연령비율, 나이비율)
      if modify_date != "":
        value_list.append(f"{page_section_dict[page_section_key]}, \"{news_url}\", \"{officeName}\", \"{title}\", \"{author_info}\", \"{create_date}\", \"{modify_date}\", \"{news_contents}\", \"{image_url}\"")
      else:
        value_list.append(f"{page_section_dict[page_section_key]}, \"{news_url}\", \"{officeName}\", \"{title}\", \"{author_info}\", \"{create_date}\", null, \"{news_contents}\", \"{image_url}\"")

      # MongoDB 용
      js_data = {}
      js_data["category_id"] = page_section_dict[page_section_key]
      js_data["category_name"] = page_section_key
      js_data["news_url"] = news_url
      js_data["company_name"] = officeName
      js_data["title"] = title
      js_data["author_info"] = author_info
      js_data["create_date"] = create_date
      js_data["modify_date"] = modify_date
      js_data["news_contents"] = news_contents
      js_data["image_url"] = image_url
      json_list.append(js_data)

#MySQL

In [ ]:
# connect_data = {"host" : "3.35.4.182", "port" : 3306, "user" : "eun",  "passwd" : "12344321"}
connect_data = {"host" : "shtestdb.duckdns.org", "port" : 13306, "user" : "cha",  "passwd" : "12344321"}
db = MySQLdb.connect(**connect_data)
curs = db.cursor()

In [ ]:
query = "use mydb_test"
result = curs.execute(query)
db.commit()

# query = "INSERT INTO Article (category_id, news_url, company_name, title, author_info, create_date, modify_date, content, image_url, comments, gendor_ratio, age_ratio) VALUES "
query = "INSERT IGNORE INTO Article (category_id, news_url, company_name, title, author_info, create_date, modify_date, content, image_url) VALUES "

send_query = query;

for value in value_list:
  send_query = query + f"({value});"

  print(send_query)

  db.cursor().execute(send_query)
  db.commit()

INSERT IGNORE INTO Article (category_id, news_url, company_name, title, author_info, create_date, modify_date, content, image_url) VALUES (101, "https://n.news.naver.com/mnews/article/016/0002194748?sid=101", "헤럴드경제", "중동 ‘오일 머니’, 글로벌 자본시장 쥐락펴락", "김우영 kwy@heraldcorp.com", "2023-09-08 10:33:01", null, "WSJ “중동 국부펀드는 ATM”미국 내 자금조달 어려워진 사이 글로벌 큰 손으로사우디아라비아 실권자인 무함마드 빈 살만 왕세자. [로이터][헤럴드경제=김우영 기자] 미국 등 주요국이 인플레이션과 전쟁을 위해 강도 높은 긴축정책을 이어가는 사이 중동 국가들이 자본시장에서 큰 손으로 군림하고 있다.7일(현지시간) 미 월스트리트저널(WSJ)에 따르면 세계 최대 자산운용사 블랙록은 사우디아라비아 수도 리야드에 투자 유치를 위한 팀을 설치할 계획이다.앞서 유명 헤지펀드인 밀레니엄 매니지먼트가 2020년 카타르 두바이에 사무실을 개설한 것을 시작으로 스타트업에 집중 투자하는 헤지펀드 엑소더스포인트캐피털 등 다수의 미국 헤지펀드들이 중동에 둥지를 틀었다. 유럽의 티케하우 캐피털과 아르디안은 아랍에미리트(UAE) 아부다비로 향했다.이들 대형 투자기관들이 중동을 향한 건 사우디나 UAE의 막대한 자금을 유치하기 위해서다.사우디 국부펀드인 사우디공공투자기금(PIF)이 주식 등 투자증권에 출자한 자금 규모는 2021년 330억달러(약 44조원)에서 2022년 560억달러(약 74조8000억원)로 껑충 뛰었다. 아부다비 국부펀드 무바달라의 주식 출자 규모는 2022년 180억달러로, 1년 새 두 배 증가했다.이미 굵직한 인수합병(M&A) 등에서 중동의 존재감은 압도적이다. 최근 무바달라는 지난 7월 투자운용사 포트리스 지분을 소프트뱅크로부터 20억

#Mongo DB

In [ ]:
mg_client = MongoClient(host = "shtestdb.duckdns.org", port = 27017, username = "cha", password = "12344321")
print(mg_client.list_database_names())

db = mg_client["cha_test"]
print(db.get_collection)

data_col = db["data_col"]
print(db.list_collection_names())

['READ_ME_TO_RECOVER_YOUR_DATA', 'admin', 'cha_test', 'config']
<bound method Database.get_collection of Database(MongoClient(host=['shtestdb.duckdns.org:27017'], document_class=dict, tz_aware=False, connect=True), 'cha_test')>
['data_col']


In [ ]:
# post_id = data_col.insert_one(json_list[0]).inserted_id
# print(post_id)

post_id = data_col.insert_many(json_list).inserted_ids
print(post_id)

[ObjectId('64f4bba6873d961cdb8021a5'), ObjectId('64f4bba6873d961cdb8021a6'), ObjectId('64f4bba6873d961cdb8021a7'), ObjectId('64f4bba6873d961cdb8021a8'), ObjectId('64f4bba6873d961cdb8021a9'), ObjectId('64f4bba6873d961cdb8021aa'), ObjectId('64f4bba6873d961cdb8021ab'), ObjectId('64f4bba6873d961cdb8021ac'), ObjectId('64f4bba6873d961cdb8021ad'), ObjectId('64f4bba6873d961cdb8021ae'), ObjectId('64f4bba6873d961cdb8021af'), ObjectId('64f4bba6873d961cdb8021b0'), ObjectId('64f4bba6873d961cdb8021b1'), ObjectId('64f4bba6873d961cdb8021b2'), ObjectId('64f4bba6873d961cdb8021b3'), ObjectId('64f4bba6873d961cdb8021b4'), ObjectId('64f4bba6873d961cdb8021b5'), ObjectId('64f4bba6873d961cdb8021b6'), ObjectId('64f4bba6873d961cdb8021b7'), ObjectId('64f4bba6873d961cdb8021b8'), ObjectId('64f4bba6873d961cdb8021b9'), ObjectId('64f4bba6873d961cdb8021ba'), ObjectId('64f4bba6873d961cdb8021bb'), ObjectId('64f4bba6873d961cdb8021bc'), ObjectId('64f4bba6873d961cdb8021bd'), ObjectId('64f4bba6873d961cdb8021be'), ObjectId('6

In [ ]:
data_col.find_one()

{'_id': ObjectId('64f4baf2873d961cdb802191'),
 'category_id': '104',
 'category_name': '세계',
 'news_url': 'https://n.news.naver.com/mnews/article/001/0014168449?sid=104',
 'officeName': '연합뉴스',
 'title': '美상무 \\"美, 中에 대응할 채찍 가지고 있다…언제든 사용 가능\\"(종합)',
 'author_info': '김경희(kyunghee@yna.co.kr)',
 'create_date': '2023-09-04 00:48:55',
 'modify_date': '2023-09-04 00:49:04',
 'news_contents': '삼성·SK하이닉스 적용 반도체 수출통제 유예 연장엔 \\"검토 중\\"\\"中과 단절 원치않아…군사적 사용 가능한 민감 반도체 한정해 수출통제\\"지나 러몬도 미 상무 장관[AP 연합뉴스 자료 사진. 재판매 및 DB 금지](워싱턴=연합뉴스) 김경희 특파원 = 최근 중국을 방문한 지나 러몬도 미 상무장관은 3일(현지시간) \\"미국은 (중국에 대한) 채찍을 가지고 있으며, 필요하다면 언제든 사용할 준비가 돼 있다\\"고 말했다.    러몬도 장관은 이날 CNN 방송에 출연, 자신의 중국 방문 성과에 대해 \\"미·중이 안정적이고 유의미한 무역 관계를 유지하는 것은 세계적 관심사\\"라며 \\"방중 기간 중국 정부의 고위 관료들과 두루 만났다\\"고 밝혔다.    러몬도 장관은 \\"그들은 우리가 다양한 수단을 가지고 있다는 것을 인지하고 있으며, 상무부의 경우 수출 통제 및 투자 규제, 관세 등이 그것들\\"이라면서 \\"우리는 채찍을 가지고 있으며, 필요하다면 언제든 사용할 준비가 돼 있다\\"고 강조했다.    최근 자신을 포함한 미국 정부 고위 인사들과 기관들의 이메일 계정이 중국 해커들에게 공격받은 데 대해선 \\"중국 카운터파트와 대화 내내 나는 매우 분명하고 직접적

In [ ]:
data_col.count_documents({})

60

In [ ]:
#data_col.delete_many({})